## Spark
---
#### Elo notes



In [20]:
import string
import json
import pickle as pk
import pandas as pd
import numpy as np

from pyspark.mllib.feature import HashingTF
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.classification import NaiveBayes

from pyspark.ml.feature import StringIndexer

from pyspark.sql import SQLContext
from pyspark.sql import Row

from pyspark.sql.functions import udf, lit, col, substring_index
from pyspark.sql.types import StringType, DoubleType

import pyspark.sql.functions as sf
import pyspark.sql.types as sparktypes

from sklearn.datasets import fetch_20newsgroups

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.porter import PorterStemmer

import re

from collections import Counter


In [9]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [10]:
# import pyspark as ps
# sc = ps.SparkContext('local[4]')
# sc

### Python - Sklearn

[20 Newsgroups](http://qwone.com/~jason/20Newsgroups/)

In [11]:
news_ = fetch_20newsgroups(subset='train')

In [12]:
categories = news_.target_names

In [13]:
categories

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [14]:
news = news_.data

In [15]:
news[2]

u'From: twillis@ec.ecn.purdue.edu (Thomas E Willis)\nSubject: PB questions...\nOrganization: Purdue University Engineering Computer Network\nDistribution: usa\nLines: 36\n\nwell folks, my mac plus finally gave up the ghost this weekend after\nstarting life as a 512k way back in 1985.  sooo, i\'m in the market for a\nnew machine a bit sooner than i intended to be...\n\ni\'m looking into picking up a powerbook 160 or maybe 180 and have a bunch\nof questions that (hopefully) somebody can answer:\n\n* does anybody know any dirt on when the next round of powerbook\nintroductions are expected?  i\'d heard the 185c was supposed to make an\nappearence "this summer" but haven\'t heard anymore on it - and since i\ndon\'t have access to macleak, i was wondering if anybody out there had\nmore info...\n\n* has anybody heard rumors about price drops to the powerbook line like the\nones the duo\'s just went through recently?\n\n* what\'s the impression of the display on the 180?  i could probably swi

### Spark - MLlib
[from Sklearn to MLlib](http://blog.xebia.fr/2015/10/08/from-scikit-learn-to-spark-ml/)


In [22]:
# Gering the categories from Python Sklearn 
cate = categories

In [23]:
cate

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [18]:
# Raw data
text_rdd = sc.wholeTextFiles('file:///Users/eloisaelias/Desktop/weeks/20_newsgroups/*').cache()

In [19]:
text_rdd.take(3)

[(u'file:/Users/eloisaelias/Desktop/weeks/20_newsgroups/talk.politics.mideast/75895',
  u'Newsgroups: talk.politics.mideast\nPath: cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu.edu!bb3.andrew.cmu.edu!news.sei.cmu.edu!cis.ohio-state.edu!zaphod.mps.ohio-state.edu!cs.utexas.edu!uunet!brunix!doorknob!hm\nFrom: hm@cs.brown.edu (Harry Mamaysky)\nSubject: Heil Hernlem \nIn-Reply-To: hernlem@chess.ncsu.edu\'s message of Wed, 14 Apr 1993 12:58:13 GMT\nMessage-ID: <HM.93Apr15112701@yoda.cs.brown.edu>\nSender: news@cs.brown.edu\nOrganization: Dept. of Computer Science, Brown University\nReferences: <1993Apr14.125813.21737@ncsu.edu>\nDate: Thu, 15 Apr 1993 16:27:01 GMT\nLines: 24\n\nIn article <1993Apr14.125813.21737@ncsu.edu> hernlem@chess.ncsu.edu (Brad Hernlem) writes:\n\n   Lebanese resistance forces detonated a bomb under an Israeli occupation\n   patrol in Lebanese territory two days ago. Three soldiers were killed and\n   two wounded. In "retaliation", Israeli and Israeli-backed forces wou

#### elo, check page 66

get the filepath

[I have to stackoverflow things](https://stackoverflow.com/questions/43209811/pyspark-how-to-access-the-values-in-a-tuple-in-a-key-tuple-rdd-python)

In [24]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [25]:
sqlc =  SQLContext(sc)

In [26]:
df = sqlc.createDataFrame(text_rdd, ['filepath', 'text'])

In [27]:
df

DataFrame[filepath: string, text: string]

In [28]:
df.show(2)

+--------------------+--------------------+
|            filepath|                text|
+--------------------+--------------------+
|file:/Users/elois...|Newsgroups: talk....|
|file:/Users/elois...|Xref: cantaloupe....|
+--------------------+--------------------+
only showing top 2 rows



In [29]:
df.select('filepath').take(1)

[Row(filepath=u'file:/Users/eloisaelias/Desktop/weeks/20_newsgroups/talk.politics.mideast/75895')]

elo: for 'text' apply substring index and separate by 'from:' and get everything in the right side to obtain only the text. 

In [30]:
# clean text : ctext
ctext = df.select(substring_index(df.text,'\nFrom:', -1).alias('ctext'))

In [31]:
ctext

DataFrame[ctext: string]

In [32]:
ctext.head()

Row(ctext=u' hm@cs.brown.edu (Harry Mamaysky)\nSubject: Heil Hernlem \nIn-Reply-To: hernlem@chess.ncsu.edu\'s message of Wed, 14 Apr 1993 12:58:13 GMT\nMessage-ID: <HM.93Apr15112701@yoda.cs.brown.edu>\nSender: news@cs.brown.edu\nOrganization: Dept. of Computer Science, Brown University\nReferences: <1993Apr14.125813.21737@ncsu.edu>\nDate: Thu, 15 Apr 1993 16:27:01 GMT\nLines: 24\n\nIn article <1993Apr14.125813.21737@ncsu.edu> hernlem@chess.ncsu.edu (Brad Hernlem) writes:\n\n   Lebanese resistance forces detonated a bomb under an Israeli occupation\n   patrol in Lebanese territory two days ago. Three soldiers were killed and\n   two wounded. In "retaliation", Israeli and Israeli-backed forces wounded\n   8 civilians by bombarding several Lebanese villages. Ironically, the Israeli\n   government justifies its occupation in Lebanon by claiming that it is \n   necessary to prevent such bombardments of Israeli villages!!\n\n   Congratulations to the brave men of the Lebanese resistance! Wit

#### Extracting IDs & Topics
[I have to stackoverflow things](https://stackoverflow.com/questions/37332434/concatenate-two-pyspark-dataframes)


[I have to stackoverflow things](https://stackoverflow.com/questions/37689878/select-array-element-from-spark-dataframes-split-method-in-same-call )

In [33]:
topic = df.select(sf.split(df.filepath, '/').getItem(6).alias('topic'))

In [34]:
topic.head()

Row(topic=u'talk.politics.mideast')

In [38]:
dfid = df.select(substring_index(df.filepath,'/',-1).alias('id'),substring_index(df.text,'\nFrom:', -1).alias('ctext'), sf.split(df.filepath, '/').getItem(6).alias('topic'))

In [39]:
dfid

DataFrame[id: string, ctext: string, topic: string]

In [40]:
dfid.head()

Row(id=u'75895', ctext=u' hm@cs.brown.edu (Harry Mamaysky)\nSubject: Heil Hernlem \nIn-Reply-To: hernlem@chess.ncsu.edu\'s message of Wed, 14 Apr 1993 12:58:13 GMT\nMessage-ID: <HM.93Apr15112701@yoda.cs.brown.edu>\nSender: news@cs.brown.edu\nOrganization: Dept. of Computer Science, Brown University\nReferences: <1993Apr14.125813.21737@ncsu.edu>\nDate: Thu, 15 Apr 1993 16:27:01 GMT\nLines: 24\n\nIn article <1993Apr14.125813.21737@ncsu.edu> hernlem@chess.ncsu.edu (Brad Hernlem) writes:\n\n   Lebanese resistance forces detonated a bomb under an Israeli occupation\n   patrol in Lebanese territory two days ago. Three soldiers were killed and\n   two wounded. In "retaliation", Israeli and Israeli-backed forces wounded\n   8 civilians by bombarding several Lebanese villages. Ironically, the Israeli\n   government justifies its occupation in Lebanon by claiming that it is \n   necessary to prevent such bombardments of Israeli villages!!\n\n   Congratulations to the brave men of the Lebanese re

In [41]:
dfid.show(2)

+-----+--------------------+--------------------+
|   id|               ctext|               topic|
+-----+--------------------+--------------------+
|75895| hm@cs.brown.edu ...|talk.politics.mid...|
|76248| waldo@cybernet.c...|talk.politics.mid...|
+-----+--------------------+--------------------+
only showing top 2 rows



[I have to stackoverflow things](https://mingchen0919.github.io/learning-apache-spark/categorical-data.html)


In [42]:
# Build the indexer
indexer = StringIndexer(inputCol= 'topic', outputCol='label')

In [43]:
# Learn the StringIndexer model : simodel
simodel = indexer.fit(dfid)

In [44]:
dftopic = simodel.transform(dfid)

In [45]:
dftopic.show()

+-----+--------------------+--------------------+-----+
|   id|               ctext|               topic|label|
+-----+--------------------+--------------------+-----+
|75895| hm@cs.brown.edu ...|talk.politics.mid...|  7.0|
|76248| waldo@cybernet.c...|talk.politics.mid...|  7.0|
|76277| C.L.Gannon@newca...|talk.politics.mid...|  7.0|
|76045| shaig@Think.COM ...|talk.politics.mid...|  7.0|
|77197| ez000281@hamlet....|talk.politics.mid...|  7.0|
|76421| amoss@shuldig.cs...|talk.politics.mid...|  7.0|
|76283| koc@rize.ECE.ORS...|talk.politics.mid...|  7.0|
|77351| sera@zuma.UUCP (...|talk.politics.mid...|  7.0|
|75406| flax@frej.teknik...|talk.politics.mid...|  7.0|
|77369| sera@zuma.UUCP (...|talk.politics.mid...|  7.0|
|76089| gt1091a@prism.ga...|talk.politics.mid...|  7.0|
|76419| adams@bellini.be...|talk.politics.mid...|  7.0|
|75892| mafifi@eis.calst...|talk.politics.mid...|  7.0|
|76284| dbd@urartu.sdpa....|talk.politics.mid...|  7.0|
|77356| murthy@ssdsun.as...|talk.politics.mid...

In [46]:
dftopic

DataFrame[id: string, ctext: string, topic: string, label: double]

In [47]:
dftopic.select('topic', 'label').distinct().show()

+--------------------+-----+
|               topic|label|
+--------------------+-----+
|comp.sys.ibm.pc.h...|  9.0|
|        misc.forsale|  8.0|
|     sci.electronics| 18.0|
|  talk.religion.misc| 10.0|
|  talk.politics.guns| 15.0|
|     rec.motorcycles| 14.0|
|  talk.politics.misc| 16.0|
|           sci.space|  1.0|
|      comp.windows.x|  6.0|
|comp.sys.mac.hard...| 11.0|
|soc.religion.chri...| 19.0|
|talk.politics.mid...|  7.0|
|    rec.sport.hockey|  0.0|
|           rec.autos| 13.0|
|             sci.med|  5.0|
|  rec.sport.baseball| 12.0|
|           sci.crypt|  3.0|
|         alt.atheism|  4.0|
|       comp.graphics|  2.0|
|comp.os.ms-window...| 17.0|
+--------------------+-----+



In [48]:
# Adding binary classification for science topics
dff_ = dftopic.select('id', 'ctext', 'topic', 'label', (substring_index(dftopic.topic,'.', 1)=='sci').alias('target'))

In [49]:
dff_

DataFrame[id: string, ctext: string, topic: string, label: double, target: boolean]

In [50]:
dff_.show(2)

+-----+--------------------+--------------------+-----+------+
|   id|               ctext|               topic|label|target|
+-----+--------------------+--------------------+-----+------+
|75895| hm@cs.brown.edu ...|talk.politics.mid...|  7.0| false|
|76248| waldo@cybernet.c...|talk.politics.mid...|  7.0| false|
+-----+--------------------+--------------------+-----+------+
only showing top 2 rows



In [51]:
dff = dff_.select('id','ctext','topic','label',dff_.target.cast('Double').alias('binary'))

In [52]:
dff.select('binary').distinct().show()

+------+
|binary|
+------+
|   0.0|
|   1.0|
+------+



In [53]:
dff.show(2)

+-----+--------------------+--------------------+-----+------+
|   id|               ctext|               topic|label|binary|
+-----+--------------------+--------------------+-----+------+
|75895| hm@cs.brown.edu ...|talk.politics.mid...|  7.0|   0.0|
|76248| waldo@cybernet.c...|talk.politics.mid...|  7.0|   0.0|
+-----+--------------------+--------------------+-----+------+
only showing top 2 rows



In [54]:
dff.printSchema()

root
 |-- id: string (nullable = true)
 |-- ctext: string (nullable = true)
 |-- topic: string (nullable = true)
 |-- label: double (nullable = true)
 |-- binary: double (nullable = true)



In [55]:
dff.head()

Row(id=u'75895', ctext=u' hm@cs.brown.edu (Harry Mamaysky)\nSubject: Heil Hernlem \nIn-Reply-To: hernlem@chess.ncsu.edu\'s message of Wed, 14 Apr 1993 12:58:13 GMT\nMessage-ID: <HM.93Apr15112701@yoda.cs.brown.edu>\nSender: news@cs.brown.edu\nOrganization: Dept. of Computer Science, Brown University\nReferences: <1993Apr14.125813.21737@ncsu.edu>\nDate: Thu, 15 Apr 1993 16:27:01 GMT\nLines: 24\n\nIn article <1993Apr14.125813.21737@ncsu.edu> hernlem@chess.ncsu.edu (Brad Hernlem) writes:\n\n   Lebanese resistance forces detonated a bomb under an Israeli occupation\n   patrol in Lebanese territory two days ago. Three soldiers were killed and\n   two wounded. In "retaliation", Israeli and Israeli-backed forces wounded\n   8 civilians by bombarding several Lebanese villages. Ironically, the Israeli\n   government justifies its occupation in Lebanon by claiming that it is \n   necessary to prevent such bombardments of Israeli villages!!\n\n   Congratulations to the brave men of the Lebanese re

In [56]:
dff.show(2)

+-----+--------------------+--------------------+-----+------+
|   id|               ctext|               topic|label|binary|
+-----+--------------------+--------------------+-----+------+
|75895| hm@cs.brown.edu ...|talk.politics.mid...|  7.0|   0.0|
|76248| waldo@cybernet.c...|talk.politics.mid...|  7.0|   0.0|
+-----+--------------------+--------------------+-----+------+
only showing top 2 rows



---
elo: add CV

## PySpark - RDD
### Tokenize, Stopwords, Stemming and lemmatization

Before any real text processing is to be done, text needs to be segmented into linguistic units such as words, punctuation, numbers, alpha-numerics, etc. This process is called tokenization.

Stop words are words which are filtered out before or after processing of natural language data (text). [Any group of words can be chosen as the stop words for a given purpose](https://raw.githubusercontent.com/stanfordnlp/CoreNLP/master/data/edu/stanford/nlp/patterns/surface/stopwords.txt). For some search engines, these are some of the most common, short function words, such as the, is, at, which, and on. In this case, stop words can cause problems when searching for phrases that include them, particularly in names such as "The Who", "The The", or "Take That".

[Stemming and lemmatization](https://nlp.stanford.edu/IR-book/html/htmledition/stemming-and-lemmatization-1.html)

In [59]:
rddlabel = dff.rdd.map(lambda x:(x['label'], x['topic'])).distinct()

In [63]:
rddlabel.take(20)

[(9.0, u'comp.sys.ibm.pc.hardware'),
 (8.0, u'misc.forsale'),
 (12.0, u'rec.sport.baseball'),
 (13.0, u'rec.autos'),
 (6.0, u'comp.windows.x'),
 (19.0, u'soc.religion.christian'),
 (7.0, u'talk.politics.mideast'),
 (2.0, u'comp.graphics'),
 (10.0, u'talk.religion.misc'),
 (15.0, u'talk.politics.guns'),
 (5.0, u'sci.med'),
 (4.0, u'alt.atheism'),
 (18.0, u'sci.electronics'),
 (11.0, u'comp.sys.mac.hardware'),
 (1.0, u'sci.space'),
 (14.0, u'rec.motorcycles'),
 (3.0, u'sci.crypt'),
 (17.0, u'comp.os.ms-windows.misc'),
 (16.0, u'talk.politics.misc'),
 (0.0, u'rec.sport.hockey')]

In [64]:
rddtext = dff.rdd.map(lambda x: (x['label'], x['ctext']))

In [65]:
rddtext.first()

(7.0,
 u' hm@cs.brown.edu (Harry Mamaysky)\nSubject: Heil Hernlem \nIn-Reply-To: hernlem@chess.ncsu.edu\'s message of Wed, 14 Apr 1993 12:58:13 GMT\nMessage-ID: <HM.93Apr15112701@yoda.cs.brown.edu>\nSender: news@cs.brown.edu\nOrganization: Dept. of Computer Science, Brown University\nReferences: <1993Apr14.125813.21737@ncsu.edu>\nDate: Thu, 15 Apr 1993 16:27:01 GMT\nLines: 24\n\nIn article <1993Apr14.125813.21737@ncsu.edu> hernlem@chess.ncsu.edu (Brad Hernlem) writes:\n\n   Lebanese resistance forces detonated a bomb under an Israeli occupation\n   patrol in Lebanese territory two days ago. Three soldiers were killed and\n   two wounded. In "retaliation", Israeli and Israeli-backed forces wounded\n   8 civilians by bombarding several Lebanese villages. Ironically, the Israeli\n   government justifies its occupation in Lebanon by claiming that it is \n   necessary to prevent such bombardments of Israeli villages!!\n\n   Congratulations to the brave men of the Lebanese resistance! With e

In [57]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [58]:
remove = set(string.punctuation)
stemmer = SnowballStemmer('english')
stop = stopwords.words('english')

In [66]:
# Normalizing text : rddntxt
# Return:= K: label, v: list of the stemmed words of the article 

rddntxt = rddtext.map(lambda (k, v): (k, v.lower())) \
                .map(lambda (k, v): (k, ''.join(char for char in v if char not in remove))) \
                .map(lambda (k, v): (k, [stemmer.stem(word) for word in v.split() if word not in stop]))


In [67]:
rddntxt.first()

(7.0,
 [u'hmcsbrownedu',
  u'harri',
  u'mamayski',
  u'subject',
  u'heil',
  u'hernlem',
  u'inreplyto',
  u'hernlemchessncsuedus',
  u'messag',
  u'wed',
  u'14',
  u'apr',
  u'1993',
  u'125813',
  u'gmt',
  u'messageid',
  u'hm93apr15112701yodacsbrownedu',
  u'sender',
  u'newscsbrownedu',
  u'organ',
  u'dept',
  u'comput',
  u'scienc',
  u'brown',
  u'univers',
  u'refer',
  u'1993apr1412581321737ncsuedu',
  u'date',
  u'thu',
  u'15',
  u'apr',
  u'1993',
  u'162701',
  u'gmt',
  u'line',
  u'24',
  u'articl',
  u'1993apr1412581321737ncsuedu',
  u'hernlemchessncsuedu',
  u'brad',
  u'hernlem',
  u'write',
  u'lebanes',
  u'resist',
  u'forc',
  u'deton',
  u'bomb',
  u'isra',
  u'occup',
  u'patrol',
  u'lebanes',
  u'territori',
  u'two',
  u'day',
  u'ago',
  u'three',
  u'soldier',
  u'kill',
  u'two',
  u'wound',
  u'retali',
  u'isra',
  u'israeliback',
  u'forc',
  u'wound',
  u'8',
  u'civilian',
  u'bombard',
  u'sever',
  u'lebanes',
  u'villag',
  u'iron',
  u'isra',


[I have to stackoverflow things](https://stackoverflow.com/questions/22350722/what-is-the-difference-between-map-and-flatmap-and-a-good-use-case-for-each )


In [68]:
# flatmap all the values
rddv = rddntxt.values().flatMap(lambda x: x)

In [69]:
rddv.take(10)

[u'hmcsbrownedu',
 u'harri',
 u'mamayski',
 u'subject',
 u'heil',
 u'hernlem',
 u'inreplyto',
 u'hernlemchessncsuedus',
 u'messag',
 u'wed']

In [70]:
# Obtain all unique words
rddvd = rddv.flatMap(lambda x: x).distinct()

In [71]:
rddvd.take(10)

[u'\x03', u'\x05', u'\x13', u'\x15', u'\x19', u'\x1b', u'1', u'3', u'5', u'7']

In [77]:
# Adding Index to the unique words
rddvdz = rddvd.zipWithIndex()

In [78]:
rddvdz.take(3)

[(u'\x03', 0), (u'\x05', 1), (u'\x13', 2)]

In [80]:
# Collect in dictionary form
dwords = rddvdz.collectAsMap()

In [81]:
len(dwords)

52

In [84]:
# Dictionary with words as keys and Index as values
words_dict = rddntxt.values().flatMap(lambda x:x).distinct().zipWithIndex().collectAsMap()

In [85]:
len(words_dict)

198673